In [1]:
from google.colab import drive
drive.mount("/content/gdrive/")

Mounted at /content/gdrive/


This code is used to find the circles inside the cropped calibration panel.
It allows us to determinate the real area and real perimeter of the chronic wounds. To crop the calibration panel, I used the bounding box that I predicted with my darknet model.
At the end, you can see the circle drawing on the cropped calibration panel. It detects approximately 50% of them. Circles not detected often correspond to blurred image or a calibration panel which is at the extremity of the image. However you can try differents parameters for the HoughCircle function and differents images treatment. I used these parameters because when the circle in the calibration panel is detected, it draws well the circle inside.

In [ ]:
!ln -s /content/gdrive/MyDrive/ /mydrive2

In [ ]:
from google.colab.patches import cv2_imshow
import numpy as np
import json
import cv2

In [ ]:
def load_predictions(file_path):
    with open(file_path, 'r') as file:
        predictions = json.load(file)
    return predictions

In [ ]:
def calculate_conversion_factor(objects, calibration_square_cm2, image_width, image_height):
    for obj in objects:
        label = obj["name"]
        confidence = obj["confidence"]
        bbox = list(obj["relative_coordinates"].values())
        if label == "calibrage":
            # Calculate the conversion factors
            area_calibration_pixels = (bbox[2] * bbox[3] * image_width * image_height)
            conversion_factor = calibration_square_cm2 / area_calibration_pixels
            return conversion_factor
    return None

In [ ]:
def calculate_real_world_area(mask, conversion_factor):
    # Count non-zero pixels (white pixels) in the mask
    area_wound_pixels = np.count_nonzero(mask)
    if conversion_factor is not None:
        area_reelle_cm2 = area_wound_pixels * conversion_factor
        return area_reelle_cm2
    else:
        return None

In [ ]:
# Given calibration details
radius_calibration_cm = 1.35
side_length_calibration_cm = 3.5
area_calibration_circle_cm2 = np.pi * (radius_calibration_cm ** 2)
area_calibration_square_cm2 = side_length_calibration_cm ** 2

# Load predictions
predictions_file_path = "/mydrive2/chronic_wounds_project/yolov4-2DB/darknet/data/my_predictions.txt"
predictions = load_predictions(predictions_file_path)

# For each detected object, measure the area in pixels
for prediction in predictions:
    image_path = "/mydrive2/chronic_wounds_project/yolov4/darknet/" + prediction["filename"]
    image = cv2.imread(image_path)
    objects = prediction["objects"]

    mask_filename = prediction["filename"].replace("JPEGImages", "Mask").replace(".jpg", ".png")
    mask_filename = f"/mydrive2/chronic_wounds_project/Segmentation:U-Net/" + mask_filename
    mask = cv2.imread(mask_filename, cv2.IMREAD_GRAYSCALE)

    # Calculate the conversion factor
    if mask is not None:
      image_height, image_width = mask.shape
      conversion_factor = calculate_conversion_factor(objects, area_calibration_square_cm2, image_width, image_height)

      # Calculate the real-world area of the wound
      area_wound_reelle_cm2 = calculate_real_world_area(mask, conversion_factor)

      # Calculate the perimeter of the shape
      contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
      perimeter_reelle_cm = 0
      for contour in contours:
        perimeter_reelle_cm += cv2.arcLength(contour, True)*conversion_factor

      if area_wound_reelle_cm2 is not None:
        # Do something with the real-world area information
        print(f"The real-world area of the wound is estimated to be {area_wound_reelle_cm2:.4f} cm².")
        print(f"The real-world perimeter of the wound is estimated to be {perimeter_reelle_cm:.4f} cm.")

    if mask is not None:
      cv2_imshow(mask)
    cv2.waitKey(0)

cv2.destroyAllWindows()  # Close all OpenCV windows when done

In [ ]:
import cv2
import numpy as np
import json

# Fonction pour calculer le rayon du cercle dans la calibration
def calculate_calibration_radius(image, calibration_coordinates, marging=10):
    # Convertissez les coordonnées relatives en coordonnées absolues
    image_height, image_width, _ = image.shape
    abs_x = int(calibration_coordinates["center_x"] * image_width - marging)
    abs_y = int(calibration_coordinates["center_y"] * image_height - marging)
    abs_radius = int(min(calibration_coordinates["width"] * image_width, calibration_coordinates["height"] * image_height) / 2 - marging)

    # Définissez la ROI autour de la calibration
    roi = image[abs_y - abs_radius:abs_y + abs_radius, abs_x - abs_radius:abs_x + abs_radius]

    # Convertissez l'image en niveaux de gris
    normalized_roi = cv2.normalize(roi, None, 0, 255, cv2.NORM_MINMAX)

    # Convertir l'image en HSV
    hsv_roi = cv2.cvtColor(normalized_roi, cv2.COLOR_BGR2HSV)

    # Définir la plage de couleur verte dans l'espace HSV
    lower_green = np.array([40, 50, 50])
    upper_green = np.array([80, 255, 255])

    # Créer un masque pour les pixels verts
    green_mask = cv2.inRange(hsv_roi, lower_green, upper_green)

    # Appliquer le masque à l'image de bord
    edges_masked = cv2.bitwise_and(roi, roi, mask=green_mask)
    gray_roi = cv2.cvtColor(edges_masked, cv2.COLOR_BGR2GRAY)

    # Appliquez l'algorithme de détection de cercle (ajustez les paramètres selon vos besoins)
    circles = cv2.HoughCircles(
        gray_roi,
        cv2.HOUGH_GRADIENT,
        dp=1.6,
        minDist=30,
        param1=50,
        param2=30,
        minRadius=int(abs_radius * 0.5),
        maxRadius=int(abs_radius * 1.3),
    )

    if circles is not None:
        circles = np.uint16(np.around(circles))
        a, b, r = circles[0][0][0], circles[0][0][1], circles[0][0][2]

        # Dessiner la circonférence du cercle.
        cv2.circle(roi, (a, b), r, (0, 255, 0), 2)

        # Dessiner un petit cercle (de rayon 1) pour montrer le centre.
        cv2.circle(roi, (a, b), 1, (0, 0, 255), 3)
        cv2_imshow(image)
        cv2_imshow(roi)
        cv2.waitKey(0)
        calibration_radius = r
        return calibration_radius
    else:
        print("Not detected")
        return None


In [ ]:
!unzip -q /mydrive2/chronic_wounds_project/yolov4-2DB/images_yolo.zip -d /mydrive2/chronic_wounds_project/yolov4-2DB/darknet/data

In [ ]:
# Given calibration details
radius_calibration_cm = 1.35
side_length_calibration_cm = 3.5

# Load predictions
predictions_file_path = "/mydrive2/chronic_wounds_project/yolov4-2DB/darknet/data/my_predictions.txt"
predictions = load_predictions(predictions_file_path)



# For each detected object, measure the area in pixels
for prediction in predictions:
    image_path = "/mydrive2/chronic_wounds_project/yolov4-2DB/darknet/" + prediction["filename"]
    image = cv2.imread(image_path)
    objects = prediction["objects"]
    for obj in objects:
      if obj["name"] == "calibration" and prediction["filename"][17:22] != "photo":
        calibration_coordinates = obj["relative_coordinates"]

        mask_filename = prediction["filename"].replace("JPEGImages", "Mask").replace(".jpg", ".png")
        mask_filename = f"/mydrive2/chronic_wounds_project/Segmentation:U-Net_2DB/" + mask_filename
        mask = cv2.imread(mask_filename, cv2.IMREAD_GRAYSCALE)
        calibration_radius = calculate_calibration_radius(image, calibration_coordinates)
        # Calculate the conversion factor
        if mask is not None:
          if calibration_radius is not None:
            conversion_factor = radius_calibration_cm/calibration_radius
            # Calculate the real-world area of the wound
            area_wound_pixels = cv2.countNonZero(mask)
            area_wound_reelle_cm2 = conversion_factor*conversion_factor*area_wound_pixels

            # Calculate the perimeter of the shape
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            perimeter_reelle_cm = 0
            for contour in contours:
              perimeter_reelle_cm += cv2.arcLength(contour, True)*conversion_factor
            cv2_imshow(mask)
            if area_wound_reelle_cm2 is not None:
              print(f"The real-world area of the wound is estimated to be {area_wound_reelle_cm2:.2f} cm².")
              print(f"The real-world perimeter of the wound is estimated to be {perimeter_reelle_cm:.2f} cm.")

          #if mask is not None:
            #cv2_imshow(mask)
          #cv2.waitKey(0)

cv2.destroyAllWindows()  # Close all OpenCV windows when done

Output hidden; open in https://colab.research.google.com to view.